# Deep Q-Network (DQN)
---
This notebook is a implementation of a DQN agent to solve OpenAI Gym's LunarLander-v2 environment running on a Windows10 machine.

### 1. Import the Necessary Packages and set global variables

In [ ]:
!pip3 install gym
!pip3 install Box2D
!pip3 install pyglet==1.5.0
!pip install gym[box2d]
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

# global variables for control flow
watch_agent = True
save_graph = True
show_graph = True

### 2. Instantiate the Environment and Untrained Agent

Initialize the environment in the code cell below.

In [25]:
# Set up Enviornment
env = gym.make('LunarLander-v2')
env.seed(0)

# Set up Agent
from dqn_agent import Agent
agent = Agent(state_size=8, action_size=4, seed=0)
state = env.reset()

# Watch an untrained agent
if watch_agent:
    done = False
    while not done:
        env.render()
        action = agent.act(state)
        state, reward, done, _ = env.step(action)

    env.close()

c:\dev\python\rl\lunar_lander\venv\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


State shape:  (8,)
Number of actions:  4


### 3. Train the Agent with DQN

In [ ]:
def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    return scores

#Train the network and save the scores
scores = dqn()

# graph and show/save the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
    
if save_graph:
    filepath = f"C:\Dev\Python\RL\Lunar_Lander\images\\baseline_Model.png"
    fig.savefig(filepath)
if show_graph:
    plt.show()

### 4. Watch a Smart Agent!

This code cell will load the trained weights from file to watch the smart agent!

In [22]:
if watch_agent:
    # load the weights from file
    agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))
    
    for i in range(5):
        state = env.reset()
        done = False
        while not done:
            env.render()
            action = agent.act(state)
            state, reward, done, _ = env.step(action)
                
    env.close()